In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jobathon-nov-2022/sample_submission.csv
/kaggle/input/jobathon-nov-2022/train.csv
/kaggle/input/jobathon-nov-2022/test.csv


In [2]:
from pandas.tseries.holiday import *

In [3]:
!pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 5.4 MB/s eta 0:00:00


In [4]:
train=pd.read_csv('/kaggle/input/jobathon-nov-2022/train.csv')
print(train.shape)

(94992, 3)


In [5]:
test=pd.read_csv('/kaggle/input/jobathon-nov-2022/test.csv')
print(test.shape)

(26304, 2)


In [6]:
train.head()

,row_id,datetime,energy
0,1,2008-03-01 00:00:00,1259.985563
1,2,2008-03-01 01:00:00,1095.541500
2,3,2008-03-01 02:00:00,1056.247500
3,4,2008-03-01 03:00:00,1034.742000
4,5,2008-03-01 04:00:00,1026.334500


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94992 entries, 0 to 94991
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   row_id    94992 non-null  int64  
 1   datetime  94992 non-null  object 
 2   energy    93092 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.2+ MB


In [8]:
train['energy'].describe()

count    93092.000000
mean      1702.995014
std        369.322103
min        866.388600
25%       1431.541100
50%       1654.465800
75%       1935.993450
max       3374.399600
Name: energy, dtype: float64

In [9]:
train['datetime']=pd.to_datetime(train['datetime'],infer_datetime_format=True)

In [10]:
train['datetime'].min(),train['datetime'].max()

(Timestamp('2008-03-01 00:00:00'), Timestamp('2018-12-31 23:00:00'))

In [11]:
hour_map={0:0,1:0,2:0,3:0,4:0,5:0,
         6:1,7:1,8:1,
         9:2,10:2,11:2,
         12:3,13:3,14:3,15:3,
         16:4,17:4,
         18:5,19:5,20:5,
         21:6,22:6,23:6}

In [12]:
train['year']=train['datetime'].dt.year
train['month']=train['datetime'].dt.month
train['day']=train['datetime'].dt.day
train['hour']=train['datetime'].dt.hour
train['quarter']=train['datetime'].dt.quarter
train['is_weekend']=train['datetime'].dt.dayofweek >4
train['day_part']=train['hour'].replace(hour_map)

In [13]:
#generate holidays feature
def gen_holiday_feat(data,start,end):
    cal = USFederalHolidayCalendar()
    holiday_dates = cal.holidays(start=start, end=end)
#     for dt in cal.holidays(start=sdt, end=edt): 
#         print (dt)
    data['is_holiday'] = False
    mask = data['datetime'].dt.date.astype('datetime64').isin(holiday_dates)
    data.loc[mask,'is_holiday']=True   
    
    data['special_holiday']=False
    mask= ((data['datetime'].dt.month==12) & (data['datetime'].dt.day>=24))  \
           | ((data['datetime'].dt.month==1) & (data['datetime'].dt.day<3)) 
    data.loc[mask,'special_holiday']=True   
        
    return holiday_dates

In [14]:
holidays= gen_holiday_feat(train,train['datetime'].dt.date.min(),train['datetime'].dt.date.max())
print(holidays)
print(train['special_holiday'].value_counts())
train['is_holiday'].value_counts()

DatetimeIndex(['2008-05-26', '2008-07-04', '2008-09-01', '2008-10-13',
               '2008-11-11', '2008-11-27', '2008-12-25', '2009-01-01',
               '2009-01-19', '2009-02-16',
               ...
               '2018-01-01', '2018-01-15', '2018-02-19', '2018-05-28',
               '2018-07-04', '2018-09-03', '2018-10-08', '2018-11-12',
               '2018-11-22', '2018-12-25'],
              dtype='datetime64[ns]', length=107, freq=None)
False    92400
True      2592
Name: special_holiday, dtype: int64


False    92424
True      2568
Name: is_holiday, dtype: int64

In [15]:
train['year'].unique()

array([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018])

In [16]:
test['datetime']=pd.to_datetime(test['datetime'],infer_datetime_format=True)

In [17]:
test['datetime'].min(),test['datetime'].max()

(Timestamp('2019-01-01 00:00:00'), Timestamp('2021-12-31 23:00:00'))

In [18]:
test['year']=test['datetime'].dt.year
test['month']=test['datetime'].dt.month
test['day']=test['datetime'].dt.day
test['hour']=test['datetime'].dt.hour
test['quarter']=test['datetime'].dt.quarter
test['is_weekend']=test['datetime'].dt.dayofweek >4
test['day_part']=test['hour'].replace(hour_map)

In [19]:
holidays= gen_holiday_feat(test,test['datetime'].dt.date.min(),test['datetime'].dt.date.max())
print(len(holidays))
print(holidays)
print(test['special_holiday'].value_counts())
test['is_holiday'].value_counts()

31
DatetimeIndex(['2019-01-01', '2019-01-21', '2019-02-18', '2019-05-27',
               '2019-07-04', '2019-09-02', '2019-10-14', '2019-11-11',
               '2019-11-28', '2019-12-25', '2020-01-01', '2020-01-20',
               '2020-02-17', '2020-05-25', '2020-07-03', '2020-09-07',
               '2020-10-12', '2020-11-11', '2020-11-26', '2020-12-25',
               '2021-01-01', '2021-01-18', '2021-02-15', '2021-05-31',
               '2021-07-05', '2021-09-06', '2021-10-11', '2021-11-11',
               '2021-11-25', '2021-12-24', '2021-12-31'],
              dtype='datetime64[ns]', freq=None)
False    25584
True       720
Name: special_holiday, dtype: int64


False    25560
True       744
Name: is_holiday, dtype: int64

In [20]:
targetcol = 'energy'

In [21]:
train[targetcol].fillna(train[targetcol].mean(),inplace=True)

Create Lag Features

In [22]:
def create_lag(data,lagno_list):
    res = pd.DataFrame()
    for i in lagno_list:
        shifted = data.shift(i)
        res=pd.concat([res,shifted],axis=1)

#     res=pd.concat([data.shift(i) for i in lagno_list],axis=1)
    res.columns=[f'lag_{i}' for i in lagno_list]
    return res

In [23]:
test.shape

(26304, 11)

In [24]:
train['istrain']=1
test['istrain']=0
combined = pd.concat([train,test],axis=0) 
#lag 1 year, 3 year, quarter, month,week
lag_df = create_lag(combined[targetcol],[24,168,720,2160,8760,26304])
combined=pd.concat([combined,lag_df],axis=1)
train=combined[combined['istrain']==1]
test=combined[combined['istrain']==0]

del combined,train['istrain'],test['istrain'],test[targetcol]
print(train.shape,test.shape)

(94992, 18) (26304, 17)


In [25]:
# cols = [col for col in train.columns if col.startswith('lag_')]
# train.drop(cols,axis=1,inplace=True)
# test.drop(cols,axis=1,inplace=True)
train.columns

Index(['row_id', 'datetime', 'energy', 'year', 'month', 'day', 'hour',
       'quarter', 'is_weekend', 'day_part', 'is_holiday', 'special_holiday',
       'lag_24', 'lag_168', 'lag_720', 'lag_2160', 'lag_8760', 'lag_26304'],
      dtype='object')

In [26]:
cols = [col for col in train.columns if col.startswith('lag_')]
target_mean = train[targetcol].mean()
for col in cols:
    train[col].fillna(target_mean,inplace=True)

Split Train into train and validation sets

In [27]:
# import cudf
# from cuml.tsa.arima import ARIMA

In [28]:
import datetime 
train_start = datetime.datetime(year=2008,month=1,day=1,hour=0)
val_start = datetime.datetime(year=2016,month=1,day=1,hour=0)
val_end = datetime.datetime(year=2018,month=12,day=31,hour=23)
# train_start = datetime.datetime(year=2017,month=1,day=1,hour=0)
# val_start = datetime.datetime(year=2018,month=1,day=1,hour=0)
# val_end = datetime.datetime(year=2018,month=12,day=31,hour=23)
# hour_mask = (train['hour']>=0)
X_val= train[(train['datetime']>=val_start) & (train['datetime']<=val_end)].copy()
X_train= train[(train['datetime']>=train_start) & (train['datetime']<val_start)].copy()
print(X_train.shape)
print(X_val.shape)
X_val.head()           

(68688, 18)
(26304, 18)


,row_id,datetime,energy,year,month,day,hour,quarter,is_weekend,day_part,is_holiday,special_holiday,lag_24,lag_168,lag_720,lag_2160,lag_8760,lag_26304
68688,68689,2016-01-01 00:00:00,2002.2470,2016,1,1,0,1,False,0,True,True,2045.0170,2274.7530,1983.0616,1878.091800,2287.2788,1479.1658
68689,68690,2016-01-01 01:00:00,1843.3870,2016,1,1,1,1,False,0,True,True,1881.1468,2118.4592,1827.3788,1702.995014,2120.5124,1375.2627
68690,68691,2016-01-01 02:00:00,1735.1178,2016,1,1,2,1,False,0,True,True,1781.7982,2020.4548,1719.9650,1627.704000,2015.9704,1307.8261
68691,68692,2016-01-01 03:00:00,1666.1970,2016,1,1,3,1,False,0,True,True,1719.5984,1936.5034,1648.2336,1564.526600,1939.9740,1261.5295
68692,68693,2016-01-01 04:00:00,1630.3924,2016,1,1,4,1,False,0,True,True,1657.1542,1895.8108,1603.2640,1541.553000,1885.3868,1228.8682


In [29]:
val_target = X_val[targetcol]
print(X_val[targetcol].isnull().sum())

0


In [30]:
# val_target.fillna(X_train[targetcol].mean(),inplace=True)

In [31]:
from sklearn.metrics import mean_squared_error

In [32]:
#compute baseline error by predicting train energy mean as the energy for all time
val_preds_baseline = np.full(len(X_val),train[targetcol].mean())

In [33]:
# print(X_train[targetcol].isnull().sum())
# X_train.fillna(X_train[targetcol].mean(),inplace=True)
# print(X_train[targetcol].isnull().sum())

In [34]:
print(val_target[0:5])

68688    2002.2470
68689    1843.3870
68690    1735.1178
68691    1666.1970
68692    1630.3924
Name: energy, dtype: float64


In [35]:
#compute error score on baseline predictions
val_score = mean_squared_error(val_target,val_preds_baseline,squared=False)
print('valid score:',val_score)

valid score: 427.29210032008933


UCM

In [36]:
X_val.shape
X_train.shape

(68688, 18)

In [37]:
import statsmodels.api as sm

In [38]:
# X_train= X_train.set_index(X_train.datetime).sort_index()
# X_val= X_val.set_index(X_val.datetime).sort_index()

In [39]:
%%time
#splitting time series to train and test subsets
y_train = X_train[targetcol].copy()
y_test = X_val[targetcol].copy()

exog_train = X_train[['is_weekend','month','hour','is_holiday','special_holiday']].astype('int')
exog_test = X_val[['is_weekend','month','hour','is_holiday','special_holiday']].astype('int')

#Unobserved Components model definition
model_UC1 = sm.tsa.UnobservedComponents(y_train,
                                        autoregressive=2,
                                        level='lldtrend',
                                        exog=exog_train,
                                        cycle=False,
                                        irregular=False,
                                        stochastic_level = False,
                                        stochastic_trend = False,
                                        stochastic_freq_seasonal = [False,False,True],
#                                         seasonal_period =24,
                                        freq_seasonal=[{'period': 24, 'harmonics': 1},
                                                       {'period': 168, 'harmonics': 1},
#                                                        {'period': 5040, 'harmonics': 1},
                                                       {'period': 8766, 'harmonics': 2}
                                                      ]
                                       )
#fitting model to train data
model_UC1res = model_UC1.fit()

#printing statsmodels summary for model
print(model_UC1res.summary())

print("")

#calculating mean absolute error and root mean squared error for in-sample prediction of model
print(f"In-sample mean absolute error (MAE): {'%.0f' % model_UC1res.mae}, In-sample root mean squared error (RMSE): {'%.0f' % np.sqrt(model_UC1res.mse)}")

#model forecast

/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency N will be used.
  self._init_dates(dates, freq)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           11     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.74971D+00    |proj g|=  3.56193D-03


 This problem is unconstrained.



At iterate    5    f=  6.71819D+00    |proj g|=  2.13437D-02

At iterate   10    f=  5.85587D+00    |proj g|=  4.18569D-03

At iterate   15    f=  5.81330D+00    |proj g|=  3.31337D-04

At iterate   20    f=  5.81257D+00    |proj g|=  1.76703D-03

At iterate   25    f=  5.81140D+00    |proj g|=  6.05556D-04

At iterate   30    f=  5.80993D+00    |proj g|=  2.92484D-04

At iterate   35    f=  5.80966D+00    |proj g|=  1.96350D-04

At iterate   40    f=  5.80938D+00    |proj g|=  2.01699D-03

At iterate   45    f=  5.80929D+00    |proj g|=  3.71018D-04

At iterate   50    f=  5.80921D+00    |proj g|=  1.06671D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tn

/opt/conda/lib/python3.7/site-packages/statsmodels/base/model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)


                                 Unobserved Components Results                                 
Dep. Variable:                                  energy   No. Observations:                68688
Model:                local linear deterministic trend   Log Likelihood             -399022.720
                                + freq_seasonal(24(1))   AIC                         798067.439
                               + freq_seasonal(168(1))   BIC                         798167.948
                   + stochastic freq_seasonal(8766(2))   HQIC                        798098.465
                                               + AR(2)                                         
Date:                                 Sun, 20 Nov 2022                                         
Time:                                         08:34:57                                         
Sample:                                     01-01-1970                                         
                                        

In [40]:
forecast_UC1 = model_UC1res.forecast(steps=26304,exog=exog_test)
#calculating root mean squared error
RMSE_UC1 = mean_squared_error(y_test,forecast_UC1,squared=False)
print('valid score:',RMSE_UC1)

valid score: 259.1740399472912


In [41]:
import joblib

In [42]:
np.save('val_forecast_UC.npy',forecast_UC1)
joblib.dump(model_UC1res,'UC_model_val.pkl',compress=True)
X_val.to_csv('X_val.csv',index=False)

In [43]:
pd.Series(forecast_UC1.describe())

count    26304.000000
mean      2063.073223
std        198.572534
min       1553.085297
25%       1921.324813
50%       2062.138065
75%       2203.910161
max       2600.708568
Name: predicted_mean, dtype: float64

In [44]:
# pred_diff = y_test.values-forecast_UC1.reset_index(drop=True)
# print(pd.Series(pred_diff).describe())
# pd.Series(pred_diff.abs()).describe()

In [45]:
# mask1 = (pred_diff <-100)
# mask =  mask1 & (X_val.reset_index()['year']==2018) & (X_val.reset_index()['month']>0)
# print(pred_diff[mask1].shape)
# print(pred_diff[mask].shape)
# X_val.reset_index().loc[mask,'hour'].value_counts()

In [46]:
# from sklearn.preprocessing import MinMaxScaler
# import tensorflow as tf
# import os

In [47]:
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.Series(diff)

In [48]:
import matplotlib.pyplot as plt

UC for Test Prediction

In [49]:
%%time
#splitting time series to train and test subsets
y_train_full = train[targetcol].copy()

exog_train_full = train[['is_weekend','month','hour','is_holiday','special_holiday']].astype('int')
exog_test_full = test[['is_weekend','month','hour','is_holiday','special_holiday']].astype('int')

#Unobserved Components model definition
model_UC_full = sm.tsa.UnobservedComponents(y_train_full,
                                        autoregressive=2,
                                        level='lldtrend',
                                        exog=exog_train_full,
                                        cycle=False,
                                        irregular=False,
                                        stochastic_level = False,
                                        stochastic_trend = False,
                                        stochastic_freq_seasonal = [False,False,True],
#                                         seasonal_period =24,
                                        freq_seasonal=[{'period': 24, 'harmonics': 1},
                                                       {'period': 168, 'harmonics': 1},
#                                                        {'period': 5040, 'harmonics': 1},
                                                       {'period': 8766, 'harmonics': 2}
                                                      ]
                                       )
#fitting model to train data
model_UC_full_res = model_UC_full.fit()

#printing statsmodels summary for model
print(model_UC_full_res.summary())

print("")

#calculating mean absolute error and root mean squared error for in-sample prediction of model
print(f"In-sample mean absolute error (MAE): {'%.0f' % model_UC_full_res.mae}, In-sample root mean squared error (RMSE): {'%.0f' % np.sqrt(model_UC_full_res.mse)}")

/opt/conda/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency N will be used.
  self._init_dates(dates, freq)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           11     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.87085D+00    |proj g|=  3.40192D-03


 This problem is unconstrained.



At iterate    5    f=  6.69119D+00    |proj g|=  2.28953D-02

At iterate   10    f=  5.90876D+00    |proj g|=  2.26628D-03

At iterate   15    f=  5.88966D+00    |proj g|=  2.16192D-04

At iterate   20    f=  5.88939D+00    |proj g|=  1.92116D-04

At iterate   25    f=  5.88715D+00    |proj g|=  2.04354D-03

At iterate   30    f=  5.88674D+00    |proj g|=  1.38083D-03

At iterate   35    f=  5.88657D+00    |proj g|=  4.95605D-05

At iterate   40    f=  5.88654D+00    |proj g|=  3.27081D-04

At iterate   45    f=  5.88653D+00    |proj g|=  7.51533D-04


/opt/conda/lib/python3.7/site-packages/statsmodels/base/model.py:606: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)



At iterate   50    f=  5.88635D+00    |proj g|=  3.18644D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   11     50     65      1     0     0   3.186D-04   5.886D+00
  F =   5.8863544414190665     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
                                 Unobserved Components Results                                 
Dep. Variable:                                  energy   No. Observations:                94992
Model:                local linear deterministic trend   Log Likelihood             -559156.581
                                + freq_seasonal(24(1))   AIC             

In [50]:
#Predict Test Set
forecast_UC_test = model_UC_full_res.forecast(steps=26304,exog=exog_test_full)

In [51]:
joblib.dump(model_UC_full_res,'UC_model_full.pkl',compress=True)

['UC_model_full.pkl']

In [52]:
print(len(test),len(forecast_UC_test))

26304 26304


In [53]:
forecast_UC_test.index

DatetimeIndex(['1970-01-01 00:00:00.000094992',
               '1970-01-01 00:00:00.000094993',
               '1970-01-01 00:00:00.000094994',
               '1970-01-01 00:00:00.000094995',
               '1970-01-01 00:00:00.000094996',
               '1970-01-01 00:00:00.000094997',
               '1970-01-01 00:00:00.000094998',
               '1970-01-01 00:00:00.000094999',
                  '1970-01-01 00:00:00.000095',
               '1970-01-01 00:00:00.000095001',
               ...
               '1970-01-01 00:00:00.000121286',
               '1970-01-01 00:00:00.000121287',
               '1970-01-01 00:00:00.000121288',
               '1970-01-01 00:00:00.000121289',
               '1970-01-01 00:00:00.000121290',
               '1970-01-01 00:00:00.000121291',
               '1970-01-01 00:00:00.000121292',
               '1970-01-01 00:00:00.000121293',
               '1970-01-01 00:00:00.000121294',
               '1970-01-01 00:00:00.000121295'],
              dtype=

In [54]:
pd.Series(forecast_UC_test).describe()

count    26304.000000
mean      2269.119901
std        239.693010
min       1685.060499
25%       2094.712455
50%       2260.765820
75%       2435.250446
max       2957.025972
Name: predicted_mean, dtype: float64

In [55]:
subm = pd.DataFrame()
subm['row_id'] = test['row_id']
subm[targetcol]=forecast_UC_test.values
subm.to_csv('submission.csv',index=False)

In [56]:
#check saved submission data
pd.read_csv('submission.csv').head()

,row_id,energy
0,94993,2047.576496
1,94994,2015.767909
2,94995,2004.353486
3,94996,1996.878382
4,94997,2002.141353


In [57]:
# import pmdarima as pm